# **Swyp Teddy Bear Club NFT Rating and Rewards Calculation**

## **1. Overview**

An **Initial NFT Offering (INO)** is a fundraising method in the cryptocurrency and blockchain ecosystem, specifically tailored for NFTs (Non-Fungible Tokens). Similar to an Initial Coin Offering (ICO), INOs allow creators, projects, or platforms to raise funds by selling a collection of NFTs to early adopters and investors.

In this context, the **Teddy Bear Club NFTs** served as **TeddySwap's NFT offering**. By minting these NFTs, owners earn rewards in **TEDY Tokens**, with the amount determined by both the rarity and quantity of the NFTs they hold. The INO was conducted in two rounds:
- [**Round 1**](https://medium.com/@TeddySwapDEX/teddy-bears-club-minting-utility-and-launch-date-23e9a4e4446d) launched on **February 17, 2023**
- [**Round 2**](https://medium.com/@TeddySwapDEX/teddy-bears-club-round-2-c209556b379f) launched on **March 8, 2023**

### *Note!*
In TeddySwap's [**initial tokenomics**](https://medium.com/@TeddySwapDEX/teddyswap-tokenomics-fcd700aea552), the market cap was set at **5,000,000,000 TEDY**. However, it was later adjusted to **8,000,000 TEDY** in a [**second announcement**](https://medium.com/@TeddySwapDEX/tedy-tokenomics-v2-fc92723ead0c). This notebook utilizes the original market cap for the calculations, scaling it to the new market cap in the final step.


### **1.1 Purpose**

This document outlines the process for calculating the NFTs' corresponding TEDY token rewards for the Teddy Bear Club NFTs across Round 1 and Round 2. Since the ranking and reward structures differ between the two rounds, calculations are performed separately for each round and then summed to determine the total rewards, where applicable. 

Feel free to download this notebook and explore it at your convenience.

---

### **1.2 NFT Information**

### *Note!*

Throughout the TEDY life cycle and the duration of the INO, several announcements were released that affected the total TEDY rewards initially determined. Below is a summary of these announcements:

1. According to the official [Teddy Bears Club](https://tbc.teddyswap.org/) website, the TEDY earnings were updated to include an additional **10% bonus** to the initial values decided in the [initial announcement](https://medium.com/@TeddySwapDEX/teddy-bears-club-minting-utility-and-launch-date-23e9a4e4446d).

### Round 1

- **Policy ID:** `ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87`
- **Total NFTs:** 804
- **Tier System:**

| **Tier** | **Rarity Range** | **Initial TEDY Allocation** | **Updated TEDY Allocation** |
|----------|-------------------|-------------------------------|-----------------------------------------|
| Tier 1   | Rarity 1 - 15     | 28,000                        | 30,800                                  |
| Tier 2   | Rarity 16 - 100   | 17,500                        | 19,250                                  |
| Tier 3   | Rarity 101 - 250  | 12,600                        | 13,860                                  |
| Tier 4   | Rarity 251 - 500  | 11,200                        | 12,320                                  |
| Tier 5   | Rarity 501 - 804  | 10,500                        | 11,550                                  |

2. The [**initial announcement**](https://medium.com/@TeddySwapDEX/teddy-bears-club-round-2-c209556b379f) regarding **Round 2** detailed tiers for ranks **1 - 9,200**. Since **2,035** NFTs were sold, the ranks were adjusted accordingly in this [**Discord announcement**](https://discord.com/channels/1053191459597201448/1053207517154508870/1095880926086570105) on the Discord server.

### Round 2

- **Policy ID:** `da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c`
- **Total NFTs:** 2,035
- **Tier System:**

| **Tier** | **Initial Rarity Range** | **Updated Rarity Range** | **Previous MarketCap TEDY** |
|----------|-------------------|------------------------------|---------------|
| Tier 1   | Rarity 1 - 100       |  Rarity 1 - 23             |       7,000         |
| Tier 2   | Rarity 101 - 999     |  Rarity 24 - 221           |      6,000          |
| Tier 3   | Rarity 1,000 - 2,999 |  Rarity 222 - 664          |        5,000        |
| Tier 4   | Rarity 3,000 - 5,999 |  Rarity 665 - 1,327        |        4,500        |
| Tier 5   | Rarity 6,000 - 9,200 |  Rarity 1,328 - 2,035      |         4,200       |


---

### **1.3 Workflow Summary**
The workflow is divided into the following steps:

1. **Fetch Data From JSON Files**  
   1.1 Load NFT data from the JSON files.

2. **NFT Rank Calculation**  
   2.1 Assign point values based on trait value rarity.  
   2.2 Compute overall rarity ranks for each NFT.

3. **NFT Rewards Calculation**  
   3.1 Load rank data from the CSV file<br>
   3.2 Count the number of occurrences of each NFT within its respective tier.<br>
   3.3 Calculate the user’s percentage share across all sold NFTs.<br>
   3.4 Determine the user’s TEDY percentage share based on the remaining available TEDY token rewards.<br>
   3.5 Compute the total TEDY token rewards using the number and rank of owned NFTs, factoring in the user’s percentage share.<br>
   3.6 Adjust the final TEDY token rewards according to the updated market cap of 8,000,000.<br>
   
---

### **1.4 Necessary Package Installations and Open Statements**



In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json"
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"

#r "nuget: Plotly.NET.Interactive, 5.0.0"
#r "nuget: Plotly.NET.ImageExport"

In [6]:
open System.Text.Json
open System.IO
open System.Net.Http
open System.Threading.Tasks
open FSharp.Data
open FSharp.Reflection
open Plotly.NET
open Plotly.NET.Interactive
open Plotly.NET.ImageExport

# **2. Workflow**

## 1. Fetch Data
The metadata for the NFTs was scraped from the official Teddy Bears Club website ([Teddy Bears Club](https://tbc.teddyswap.org/)) and [CNFT Tools](https://cnft.tools/teddybearclub2) and stored in JSON files. Please refer to the [`webscraper.ipynb`](./webscraper.ipynb) to see how we did it, and try it out yourself. Additionally, refer to [`attributecounter.ipynb`](./attributecounter.ipynb) for the tallying of trait value occurences.

In this section, we will parse the JSON data to extract information about the NFTs and the occurrence of their traits. This data will serve as the foundation for calculating rankings and determining rewards in the subsequent sections. Using the data from the JSON files, we were also able to compute for the NFTs' rarity score and rarity rank. For efficiency, we then created a CSV file mapping subjects to their rank, which will also be extracted in this section.

---

#### **Types**:
The NFT and NFTTraits types are partial representations of the data, as only the fields relevant to ranking and reward calculations have been included.

- **`NFT`**: Represents an individual NFT, focusing on its identifiers and traits.

- **`NFTTraits`**: Captures occurrence counts of trait values across NFTs.

#### **Function**:
1. **`loadNFTs<'T>`**: 
- Loads and deserializes JSON data into a specified type:
- **Parameters**: 
     - `filePath (string)`: the path to the JSON file.
- **Returns**:
    - Parsed data of type `'T`

--- 

#### **Variables**:<br>
**`nftRound1NFTS`**: <br>
- (NFT list): NFTs from Round 1 <br>

 **`nftRound2NFTS`**:  <br>
- (NFT list):  NFTs from Round 2 <br>

 **`nftRound1Traits`**: <br>
- (NFTTraits): Trait value counts for Round 1 NFTs <br>
    
 **`nftRound2Traits`**:  <br>
- (NFTTraits): Trait value counts for Round 2 NFTs<br>

---

### 1.1 Fetch NFT Data From JSON

In [7]:
type NFT = {
    assetName: string
    encodedName: string
    Background: string
    Bear: string
    Clothes: string
    Face: string
    Handheld: string
    Head: string
    Skins: string
}

type NFTTraits = {
    Background: Map<string, int>
    Bear: Map<string, int>
    Clothes: Map<string, int>
    Face: Map<string, int>
    Handheld: Map<string, int>
    Head: Map<string, int>
    Skins: Map<string, int>
}

In [8]:
let loadNfts<'T> (filePath: string) : 'T =
    let jsonContent = File.ReadAllText(filePath)
    JsonSerializer.Deserialize<'T>(jsonContent)

In [9]:
let nftRound1NFTS = loadNfts<NFT list> "./data/collection_1.json"
let nftRound2NFTS = loadNfts<NFT list> "./data/collection_2.json"

In [10]:
let nftRound1Traits = loadNfts<NFTTraits> "./data/collectiontraits_1.json"
let nftRound2Traits = loadNfts<NFTTraits> "./data/collectiontraits_2.json"

## **2. NFT Rank Calculation** 
Each NFT has traits such as Background, Bear, Clothing, and more, with each trait having specific values. For example, the Background trait can have values like Space, Lighthouse, or Sunflowers. The overall rarity score and rank of an NFT are determined by the rarity of its trait values.

Therefore, the rarity score of an NFT is calculated by assessing the rarity of its trait values. First, we determine the percentage of NFTs that share each specific trait value by dividing its number of occurrences by the total NFTs in the round. Next, we subtract this percentage from 100 to assign higher points to rarer traits. Finally, the total rarity score is obtained by summing the points across all trait categories (e.g., Background, Hair, Clothes). The score is rounded to two decimal places for consistency, ensuring that the rarer the NFT, the higher its final rarity score.

$\text{Trait Score} = \text{round}\left(100.0 - \left(\frac{\text{Trait Frequency}}{\text{Total NFTs}}\right) \cdot 100.0, 2\right)$

$\text{Total Rarity Score} = \sum_{i=1}^{7} T_i$

---
### 2.1 Assign point values based on trait value rarity.   
Utilizing the Trait Score formula, we compute each trait value's score. We then store those scores in a Map of traits and trait values. 

#### **Function**:
1. **`processNFTTraits`**: 
- Calculates rarity scores for each trait value given an NFTTraits record and the total number of NFTs in the round.
- **Parameters**:<br>
    - **`nftTraits`**: (NFTTraits): The trait value occurrence data for a round.
        
    - **`totalNfts`**: (float): The total number of NFTs in the round.
- **Returns**: <br>
    - A Map<string, Map<string, float>> of traits, their values, and the trait value's number of occurrences

#### **Variables**:
**`r1TotalNfts`**: 
- (float): The total number of NFTs in Round 1 (804).<br>

**`r2TotalNfts`**: 
- (float): The total number of NFTs in Round 2 (2035).<br>

**`r1ScoredTraits`**: 
- (Map<string, Map<string, float>>): A map of scored trait values for Round 1 NFTs.<br>

**`r2ScoredTraits`**: 
- (Map<string, Map<string, float>>): A map of scored trait values for Round 2 NFTs.<br>


In [11]:
let r1TotalNfts = 804.0
let r2TotalNfts = 2035.0

In [12]:
let processNFTTraits (nftTraits: NFTTraits) (totalNfts: float) =
    let recordType = typeof<NFTTraits>
    FSharp.Reflection.FSharpType.GetRecordFields(recordType)
    |> Array.fold (fun acc field ->
        let fieldValue = field.GetValue(nftTraits) :?> Map<string, int>
        let traitScores = 
            fieldValue |> Map.map (fun key value ->
                Math.Round(100.0 - (float value / float totalNfts) * 100.0, 2)
            )
        Map.add field.Name traitScores acc
    ) Map.empty

In [13]:
let r1ScoredTraits = processNFTTraits nftRound1Traits r1TotalNfts
let r2ScoredTraits = processNFTTraits nftRound2Traits r2TotalNfts

### *Illustrating the Distribution of Trait Scores*: 
This section visualizes the percentages of trait occurrences alongside their corresponding scores. The createCharts function processes the trait data and generates doughnut charts that display both the raw values and their proportions.

In [14]:
let createCharts (traitScores: Map<string, Map<string, float>>) = 
    traitScores
    |> Map.fold( fun acc title labelsAndValues ->
        let labels,values = 
            labelsAndValues
            |> Map.toList
            |> List.unzip

        let combinedText = 
            List.map2 (fun label value -> sprintf "%s: %.2f" label value) labels values

        let chart = 
            Chart.Doughnut(values = values, Labels = labels, Hole = 0.5, MultiText = combinedText)
            |> Chart.withTitle title
            |> Chart.withLayout(
                Layout.init(
                    Width = 1200,
                    Height = 1000
                )
            )
                    
        chart :: acc
    )[]
    |> List.rev

let chartRound1 = createCharts r1ScoredTraits 
let chartRound2 = createCharts r2ScoredTraits

### Teddy Bear Club Round 1 Trait Value Scores

<img src="./assets/charts1/doughnut1.png" alt="Round 1 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts1/doughnut2.png" alt="Round 1 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts1/doughnut3.png" alt="Round 1 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts1/doughnut4.png" alt="Round 1 Doughnut Charts" width="1400" height="800">

In [ ]:
for chart in chartRound1 do
    display(chart |> Chart.withSize(1400,1400))

### Teddy Bears Club Round 2 Trait Value Scores

<img src="./assets/charts2/doughnut1.png" alt="Round 2 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts2/doughnut2.png" alt="Round 2 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts2/doughnut3.png" alt="Round 2 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts2/doughnut4.png" alt="Round 2 Doughnut Charts" width="1400" height="800">

In [ ]:
for chart in chartRound2 do
    display(chart |> Chart.withSize(1400,1400)) 

### 2.2 Compute overall rarity ranks for each NFT. 
Using the information from this step, we also created the CSV mapping NFTs to their ranks for easier access and computation. It will be used below in computing the the TEDY token rewards. Please refer to [`rankdata.ipynb`](./rankdata.ipynb) regarding the creation of the CSV file.

#### **Function**:
1. **`computeNFTRank`**: 
- Computes the rarity rank of NFTs based on their traits and scores.
- **Parameters**:
    - **`nftList`**: (NFT list): The list of NFTs to rank.
    - **`nftTraitScores`**: (Map<string, Map<string, float>>): Trait value scores computed above.

#### **Variables**:
**`rankedNFTsRound1`**: 
- (list): Ranks Round 1 NFTs and stores the asset's assetName, encodedName, and rarityScore.

**`rankedNFTsRound2`**: 
- (list): Ranks Round 2 NFTs with the same structure as above.


In [15]:
let computeNFTRank (nftList: list<NFT>) nftTraitScores = 
    nftList
    |> List.map (fun nft ->
        let rarityScore =  
            [
                ("Background", nft.Background)
                ("Bear", nft.Bear)
                ("Clothes", nft.Clothes)
                ("Face", nft.Face)
                ("Handheld", nft.Handheld)
                ("Head", nft.Head)
                ("Skins", nft.Skins)
            ]
            |> List.fold (fun acc (attrName, attrValue) ->  
                match Map.tryFind attrName nftTraitScores with
                | Some traitMap ->  
                    acc + (Map.tryFind attrValue traitMap |> Option.defaultValue 0.0)
                | None -> acc 
            ) 0.0 
        (nft.assetName, nft.encodedName, rarityScore) 
    )
    |> List.sortBy (fun (_, _, rarityScore) -> -rarityScore)
    |> List.mapi (fun index (assetName, encodedName, rarityScore) -> 
    (index + 1, assetName, encodedName, rarityScore))

In [16]:
let rankedNFTsRound1 = computeNFTRank nftRound1NFTS r1ScoredTraits
let rankedNFTsRound2 = computeNFTRank nftRound2NFTS r2ScoredTraits

## **3. NFT Rewards Calculation** 
Calculating a user's TEDY rewards takes into account the NFTs they own, their respective round each NFT belongs to, and their individual ranks within those rounds. The functions below will identify the tier of a subject, based upon the ranking calculations done above. Additionally, the calculation considers the user's proportional share of unsold NFTs - so the number of NFTs a user holds is taken into consideration as well.

---
### **3.1 Fetch Rank Data From CSV**
Load NFT rank data from the [`collectionranks.csv`](./data/collectionranks.csv) file in the `data` folder.  Using the calculations below, we computed the rank of each subject and then created a CSV to map those values. If you wish to read the code or recreate the CSV file's creation, refer to [`rankData.ipynb`](./rankdata.ipynb).

#### **Type**:
- **`NFTRecord`**: Represents an individual NFT record from the CSV.

#### **Functions**:

1. **`parseCsv`**:
- Parses a CSV file into a list of `NFTRecord` entries.
- **Parameters**:
   - `filePath (string)`: The path to the CSV file.
- **Returns**:  
   - A `list<NFTRecord>` containing records with `Name`, `Subject`, `Rank`, and `Score`.

2. **`createSubjectToRankMap`**:
- Creates a mapping of `Subject`(Policy Id + AssetName) to `Rank`.
- **Parameters**:
   - `records (list<NFTRecord>)`: A list of `NFTRecord` entries.
- **Returns**:  
   - A `Map<string, int>` mapping subjects to their ranks.

3. **`createNameToRankMap`**:
- Creates a mapping of `Name` (i.e. TeddyBearsClub178) to `Rank`.
- **Parameters**:
   - `records (list<NFTRecord>)`: A list of `NFTRecord` entries.
- **Returns**:  
   - A `Map<string, int>` mapping names to their ranks.

---

#### **Variables**:

1. **`csvPath`**:  
   - `(string)`: The file path to the CSV file containing NFT rank data.

2. **`nftDataList`**:  
   - `(list<NFTRecord>)`: A list of parsed `NFTRecord` entries created from the CSV file.

3. **`subjectRankData`**:  
   - `(Map<string, int>)`: A mapping of `Subject` to `Rank` created using `createSubjectToRankMap`.

4. **`nameRankData`**:  
   - `(Map<string, int>)`: A mapping of `Name` to `Rank` created using `createNameToRankMap`.


In [17]:
type NFTRecord = {
    Name: string
    Subject: string
    Rank: int
    Score: float
}

In [18]:
 let parseCsv (filePath: string) =
    File.ReadAllLines(filePath)
    |> Array.skip 1
    |> Array.map (fun line ->
        let parts = line.Split(',')
        {
            Name = parts.[0]
            Subject = parts.[1]
            Rank = int parts.[2]
            Score = float parts.[3]
        }
    )
    |> List.ofArray

In [19]:
let createSubjectToRankMap (records: NFTRecord list) =
    records
    |> List.map (fun record -> record.Subject, record.Rank)
    |> Map.ofList

let createNameToRankMap (records: NFTRecord list) =
    records
    |> List.map (fun record -> record.Name, record.Rank)
    |> Map.ofList

In [20]:
let csvPath = "./data/collectionranks.csv"
let nftDataList = parseCsv csvPath
let subjectRankData = createSubjectToRankMap  nftDataList 
let nameRankData = createNameToRankMap  nftDataList 

### **3.2 Identify a Subject's Tier**
Given a subject, composed of the NFT's policy id and asset name, we categorize the NFT into its respective tier based on its round and rank.

#### **Functions**:
1. **`categorizeNftRound1ByTier`**:
- Categorizes the Round 1 NFT according to its Tier.
- **Parameters**:
  - `subject` (string): The identifier of the NFT.
  - `rank` (int): The rarity rank of the NFT.

2. **`categorizeNftRound2ByTier`**:
- Categorizes the Round 2 NFT according to its Tier.
- **Parameters**:
  - `subject` (string): The identifier of the NFT.
  - `rank` (int): The rarity rank of the NFT.

3. **`computeBaseRewards`**:
- Given a subject, obtains the TEDY rewards allocated to the subject's tier.
- This function calls `categorizeNftRound1ByTier` or `categorizeNftRound2ByTier`.
- **Parameters**:
  - `subject` (string): The identifier of the NFT.
  - `rankData` (Map<string, int>): A mapping of subjects to their rank.
- **Returns**:
  - `baseRewards` (int): The rewards allocated to the given subject.
---

#### **Variables**:
 **`round1PolicyId`**:
   - (string): the unique policy ID for identifying NFTs from Round 1.

 **`round2PolicyId`**:
   - (string): the unique policy ID for identifying NFTs from Round 2.

**`baseReward`**:
   - (int): the amount of TEDY rewards allocated to the NFT.

In [21]:
let round1PolicyId = "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87"
let round2PolicyId = "da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c"

In [22]:
let categorizeNftRound1ByTier subject rank  =
    if rank <= 15 then 38000
    elif rank <= 100 then 19250
    elif rank <= 250 then 13860
    elif rank <= 500 then 12320
    elif rank <= 804 then 11550
    else 0  

In [23]:
let categorizeNftRound2ByTier subject rank =
    if rank <= 23 then 7000
    elif rank <= 221 then 6000
    elif rank <= 664 then 5000
    elif rank <= 1327 then 4500
    elif rank <= 2035 then 4200
    else 0

In [24]:
let computeBaseRewards (subject: string) (rankData: Map<string, int>) =
    let baseReward =
        if subject.Contains(round1PolicyId) then
            categorizeNftRound1ByTier subject subjectRankData[subject] 
        elif subject.Contains(round2PolicyId) then
            categorizeNftRound2ByTier subject subjectRankData[subject]
        else
            printfn "Unknown NFT policyId for %s" subject
            0
    baseReward

### Computing an NFT's Rank and Reward with Alice’s Teddy Bear

Now that we've built the functions to calculate NFT rankings and created the functions to identify an NFT's TEDY rewards, it's time to see these computations in action.

Let’s consider Alice, a proud owner of a unique NFT: Teddy Bears Club #5731. Naturally, Alice is curious to find out the rank and reward allocation of her beloved Teddy Bear. Using the tools and functions we’ve developed, we can help Alice uncover this information.

Below, we’ll use the code to determine the rank and tier of #5731 and show Alice where her NFT stands in the broader collection. Later on, we will also calculate for the TEDY tokens Alice can receive. Let’s dive in!

<img src="./assets/tbc5731.png" alt="Teddy Bears Club #5731" width="400" height="400">


In [47]:
let ownedNFT = "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d8754656464794265617273436c756235373331"

We then call `computeBaseRewards` to identify #5731's tier.

In [48]:
let baseReward =  computeBaseRewards ownedNFT subjectRankData

In [49]:
printfn "Asset: %s" ownedNFT
printfn "Asset's Rank: %d" subjectRankData[ownedNFT]
printfn "Asset's TEDY Reward: %d" baseReward

Asset: ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d8754656464794265617273436c756235373331
Asset's Rank: 162
Asset's TEDY Reward: 13860


### **3.3 Calculate Portion of TEDY rewards from Surplus TEDY**

As stated in the [Round 2 announcement](https://medium.com/@TeddySwapDEX/teddy-bears-club-round-2-c209556b379f), NFT holders receive an additional allocation of TEDY tokens from the pool of unsold NFTs. The formula below calculates the additional TEDY rewards per NFT owned by a user.

Where:

- **Total INO TEDY Allocation:**  
  The total amount of TEDY designated for Initial NFT Offering (INO) rewards.  
  $\text{Total INO TEDY Allocation} = 5,000,000,000 \times 4.5\% = 2,250,000,000 \text{ TEDY}$
  
- **Round 1 Bonus:**  
  The bonus multiplier applied to NFTs sold in Round 1.  
 $\text{Round 1 Bonus} = 2.75$

- **Round 1 NFTs Sold:**  
  The total number of NFTs sold in Round 1.  
  $\text{Round 1 NFTs Sold} = 804$

- **Round 2 NFTs Sold:**  
  The total number of NFTs sold in Round 2.  
  $\text{Round 2 NFTs Sold} = 2,035$

- $\text{Tier Tokens}_i$:  
  The number of TEDY tokens allocated per NFT in Tier \(i\).

- $\text{Count of Tier}_i$:
  The number of NFTs in Tier \(i\).

#### **Calculations**

1. **Calculate Distributed Tokens:**

   This represents the total TEDY tokens already distributed based on the tiers of NFTs sold in both Round 1 and Round 2.

   $\text{Distributed Tokens} = \sum_{i=1}^{5} (\text{Round 1 Tier Tokens}_i \times \text{Count of Tier}_i) + \sum_{i=1}^{5} (\text{Round 2 Tier Tokens}_i \times \text{Count of Tier}_i)$

2. **Determine Available Token Rewards:**

   Subtract the distributed tokens from the total INO TEDY allocation to find the remaining tokens available for surplus distribution.

   $\text{Available Token Rewards} = \text{Total INO TEDY Allocation} - \text{Distributed Tokens}$

3. **Compute TEDY Surplus per NFT:**

   Calculate the additional TEDY reward per NFT by dividing the available token rewards by the effective total number of NFTs, considering the bonus multiplier for Round 1 NFTs.

   $\text{TEDY Surplus per NFT} = \frac{\text{Available Token Rewards}}{(\text{Round 1 NFTs Sold} \times \text{Round 1 Bonus}) + \text{Round 2 NFTs Sold}}$

---

#### **Functions**:
1. **`calculateSurplusTedyPerNFT`**:
- Computes the additional TEDY rewards per NFT owned by a user.
- **Parameters**:
   - None explicitly passed, but it relies on:
      - `availableTokenRewards` (array) - The total amount of TEDY rewards available.
      - `round1Bonus` (float) - The bonus multiplier for Round 1 NFTs.
      - `round1NFTsSold` (int) - Total number of NFTs sold in Round 1.
      - `round2NFTsSold` (int)  Total number of NFTs sold in Round 2.
- **Returns**:
   - 

---

#### **Variables**:
**`round1Bonus`**:
   - (float): represents the bonus multiplier applied to Round 1 NFTs.

**`round1NFTsSold`**:
   - (int): represents the total NFTs minted for Round 1.

**`round2NFTsSold`**:
   - (int): epresents the total NFTs minted for Round 2.
   <br>

**`round1Tier1`, `round1Tier2`, `round1Tier3`, `round1Tier4`, `round1Tier5`**:
   - Tokens allocated per NFT for each tier in Round 1:
     - **round1Tier1**: `30800`
     - **round1Tier2**: `19250`
     - **round1Tier3**: `13860`
     - **round1Tier4**: `12320`
     - **round1Tier5**: `11550`

**`round2Tier1`, `round2Tier2`, `round2Tier3`, `round2Tier4`, `round2Tier5`**:
   - Tokens allocated per NFT for each tier in Round 2:
     - **round2Tier1**: `7000`
     - **round2Tier2**: `6000`
     - **round2Tier3**: `5000`
     - **round2Tier4**: `4500`
     - **round2Tier5**: `4200`

**`round1CountOfTier1`, `round1CountOfTier2`, `round1CountOfTier3`, `round1CountOfTier4`, `round1CountOfTier5`**:
   - The number of NFTs held in each tier:
     - **countOfTier1**: `15`
     - **countOfTier2**: `85`
     - **countOfTier3**: `150`
     - **countOfTier4**: `250`
     - **countOfTier5**: `304`

**`round2CountOfTier1`, `round2CountOfTier2`, `round2CountOfTier3`, `round2CountOfTier4`, `round2CountOfTier5`**:
   - The number of NFTs held in each tier:
     - **countOfTier1**: `23`
     - **countOfTier2**: `198`
     - **countOfTier3**: `443`
     - **countOfTier4**: `663`
     - **countOfTier5**: `708`

**`maxRoundTwoTedyTokens`**:
   - (int): The maximum number of TEDY tokens allocated for Round 2.
   - **Value**: `2250000000`

**`distributedTokens`**:
   - (int): The total tokens distributed across all both rounds.

**`availableTokenRewards`**:
   - (float): The remaining TEDY tokens available for INO rewards after distribution.


In [50]:
let round1Bonus = 2.75
let round1NftsSold = 804.0
let round2NftsSold = 2035.0

In [53]:
let round1Tier1 = 30800
let round1Tier2 = 19250
let round1Tier3 = 13860
let round1Tier4 = 12320
let round1Tier5 = 11550

let round1CountOfTier1 = 15
let round1CountOfTier2 = 85
let round1CountOfTier3 = 150
let round1CountOfTier4 = 250
let round1CountOfTier5 = 304

let round2Tier1 = 7000
let round2Tier2 = 6000
let round2Tier3 = 5000
let round2Tier4 = 4500
let round2Tier5 = 4200

let round2CountOfTier1 = 23
let round2CountOfTier2 = 198
let round2CountOfTier3 = 443
let round2CountOfTier4 = 663
let round2CountOfTier5 = 708

let totalTokenRewards = 2250000000.0

In [54]:
let round1Counts = [| round1CountOfTier1; round1CountOfTier2; round1CountOfTier3; round1CountOfTier4; round1CountOfTier5 |]
let round1Tiers = [| round1Tier1; round1Tier2; round1Tier3; round1Tier4; round1Tier5 |]
let round2Counts = [| round2CountOfTier1; round2CountOfTier2; round2CountOfTier3; round2CountOfTier4; round2CountOfTier5 |]
let round2Tiers = [| round2Tier1; round2Tier2; round2Tier3; round2Tier4; round2Tier5 |]

In [55]:
let distributedTokens = 
    (Array.zip round1Counts round1Tiers |> Array.sumBy (fun (count, tier) ->  count *  tier)) +
    (Array.zip round2Counts round2Tiers |> Array.sumBy (fun (count, tier) ->  count *  tier))


In [56]:
let availableTokenRewards =  totalTokenRewards - float distributedTokens

In [57]:
let calculateSurplusTedyPerNFT = 
    let surplusReward = availableTokenRewards/(round1NftsSold * round1Bonus + round2NftsSold)
    Math.Ceiling(surplusReward)

In [58]:
let surplusReward = calculateSurplusTedyPerNFT

## **3.4 Calculate Total TEDY Rewards** 
In this step, we will use the `loadNFTs` function from the first step to parse JSON data that we have queried from the CardanoScan API. Refer to [`assetapi.ipynb`](./assetsapi.ipynb) to see the process and test it out as well - allowing you to calculate the total TEDY rewards of a certain address. We will then utilize the `computeBaseRewards` function and `surplusReward` to compute the total Tedy rewards.

---

**Types**:

**Functions**:

**Variables**:


In [70]:
type Token = {
    policyId: string
    assetName: string
    fingerprint: string
    assetId: string
    balance: string
}

type Response = {
    tokens: Token[]
}

In [77]:
let calculateTotalRewards (filePath: string) (round1PolicyId: string) (round2PolicyId: string) (subjectRankData: Map<string, int>) (surplusReward: float) =
    let addressNfts = loadNfts<Response> filePath

    let nftList =
        addressNfts.tokens
        |> Array.map (fun token -> token.assetId)
        |> Array.toList

    let rewardsList =
        [ for nft in nftList do
            if nft.Contains(round1PolicyId) then
                yield computeBaseRewards nft subjectRankData
            elif nft.Contains(round2PolicyId) then
                yield computeBaseRewards nft subjectRankData ]

    let totalNftRewards = rewardsList |> List.sum
    let numberOfNfts = rewardsList |> List.length

    let totalTedyRewards = totalNftRewards + ((int)surplusReward * numberOfNfts)

    totalTedyRewards


In [79]:
let totalTedyRewards = calculateTotalRewards "./data/collectionassets.json" round1PolicyId round2PolicyId subjectRankData surplusReward
printfn "Total Rewards of Address: %d" totalTedyRewards


Total Rewards of Address: 529332


### **3.5 Compute Total TEDY Allocation**
Use the number and rank of the user's owned NFTs, along with the calculated percent share above, to compute the total TEDY tokens they are eligible to receive.

Where:
- NFT Rarity Rank: The number of owned NFTs for each rank.
- Rank TEDY Value: The TEDY value assigned to the rank.

$\text{Base TEDY} = \sum_{i=1}^{5} (\text{NFT Rarity Rank Count}_i \times \text{Rank TEDY Value}_i)$<br>

$\text{Round 1 Total TEDY Allocation} = \text{Base TEDY} + \text{Round 1 TEDY Share}$

$\text{Round 2 Total TEDY Allocation} = \text{Base TEDY} + \text{Round 2 TEDY Share}$

---

Calculate the total TEDY tokens earned by the user in Round 1 and Round 2, combining tokens from held NFTs across tiers and additional rewards.

#### **Functions**:
1. **`calculateTedyTokensTotal`**:
- Computes the total TEDY tokens earned in Round 1 and Round 2, including tokens from held NFTs in each tier and approximate TEDY shares.
- **Parameters**:
   - None explicitly passed, but it relies on:
      - `round1NFTsHeldByRank`: An array of counts of NFTs held by the user for each tier in Round 1.
      - `round2NFTsHeldByRank`: An array of counts of NFTs held by the user for each tier in Round 2.
      - `round1TedyShare`: Approximate TEDY rewards for Round 1 based on percent share.
      - `round2TedyShare`: Approximate TEDY rewards for Round 2 based on percent share.
      - `round1Tier1`, `round1Tier2`, ..., `round2Tier5`: Tokens allocated per NFT for each tier in Round 1 and Round 2.
- **Returns**:
   - A tuple: (`round1TotalTedyTotal`, `round2TotalTedyTotal`).

---

#### **Variables**:
**`round1NFTsHeldByRank`**:
   - array of counts of NFTs held by the user for each tier in Round 1.
   - **Structure**: `[countTier1, countTier2, countTier3, countTier4, countTier5]`.

**`round2NFTsHeldByRank`**:
   - An array of counts of NFTs held by the user for each tier in Round 2.
   - **Structure**: `[countTier1, countTier2, countTier3, countTier4, countTier5]`.

**`round1TedyShare`**:
   - Approximate TEDY rewards for Round 1 based on percent share.
   - **Type**: Integer.

**`round2TedyShare`**:
   - Approximate TEDY rewards for Round 2 based on percent share.
   - **Type**: Integer.

**`round1Tier1`, `round1Tier2`, `round1Tier3`, `round1Tier4`, `round1Tier5`**:
   - Tokens allocated per NFT for each tier in Round 1.

**`round2Tier1`, `round2Tier2`, `round2Tier3`, `round2Tier4`, `round2Tier5`**:
   - Tokens allocated per NFT for each tier in Round 2.

**`round1TotalTedy`**:
   - Total TEDY tokens from held NFTs in Round 1, calculated as:

**`round2TotalTedy`**:
   - Total TEDY tokens from held NFTs in Round 2, calculated as:

**`round1TotalTedyTotal`, `round2TotalTedyTotal`**:
   - Total TEDY tokens for Round 1 and Round 2, including additional rewards:


In [34]:
let calculateTedyTokensTotal = 
    let round1TotalTedy = (round1NFTsHeldByRank[0] * round1Tier1) + (round1NFTsHeldByRank[1] * round1Tier2) + (round1NFTsHeldByRank[2] * round1Tier3) + (round1NFTsHeldByRank[3] * round1Tier4) + (round1NFTsHeldByRank[4] * round1Tier5)
    let round2TotalTedy = (round2NFTsHeldByRank[0] * round2Tier1) + (round2NFTsHeldByRank[1] * round2Tier2) + (round2NFTsHeldByRank[2] * round2Tier3) + (round2NFTsHeldByRank[3] * round2Tier4) + (round2NFTsHeldByRank[4] * round2Tier5)

    let round1TotalTedyTotal = round1TotalTedy + round1TedyShare
    let round2TotalTedyTotal = round2TotalTedy + round2TedyShare
    (round1TotalTedyTotal,round2TotalTedyTotal)

In [35]:
let round1ApproximateTotalTedy,round2ApproximateTotalTedy = calculateTedyTokensTotal

### **5. Adjust Rewards for Market Cap**
Adjust the final TEDY reward allocation based on the updated market cap of 8,000,000 to ensure alignment with the revised tokenomics.

Where:
- NFT Rarity Rank: The number of owned NFTs for each rank.
- Rank TEDY Value: The TEDY value assigned to the rank.

$\text{Final Round Reward} = (\text{Total TEDY Allocation} * \text{Current Marketcap}) / \text{Previous Market Cap}$

---

Adjust the total TEDY rewards for Round 1 and Round 2 based on the change in market capitalization. The adjustment ensures that rewards are proportional to the current market cap compared to a past market cap.

#### **Functions**:
1. **`marketcapAdjustment`**:
- Adjusts the reward amount based on the ratio of the current market cap to the past market cap.
- **Parameters**:
   - `rewards` (float): The original reward amount to be adjusted.
- **Returns**:
   - (int): The adjusted reward amount.

---

#### **Variables**:
**`pastMarketCap`**:
   - The previous market cap of the token, used as a reference for adjustments.

**`currentMarketCap`**:
   - The current market cap of the token, used to scale rewards.

**`round1ApproximateTotalTedy`**:
   - The total TEDY tokens for Round 1 before market cap adjustment.

**`round2ApproximateTotalTedy`**:
   - The total TEDY tokens for Round 2 before market cap adjustment

**`totalTedyRound1`**:
   - The adjusted total TEDY tokens for Round 1

**`totalTedyRound2`**:
   - The adjusted total TEDY tokens for Round 2

**`totalTedyReward`**:
   - The combined adjusted TEDY tokens for both rounds


In [36]:
let pastMarketCap = 5000000000.0
let currentMarketCap = 8000000.0
let marketcapAdjustment (rewards: float) = 
    let finalRoundReward = (rewards*currentMarketCap)/pastMarketCap
    Math.Ceiling(finalRoundReward)

In [37]:
let totalTedyRound1 = int(marketcapAdjustment round1ApproximateTotalTedy)
let totalTedyRound2 = int(marketcapAdjustment round2ApproximateTotalTedy)
let totalTedyReward = totalTedyRound1 + totalTedyRound2

### Alice's Total TEDY Rewards

Now that we’ve crafted the functions to calculate rewards, it’s time to discover the value of Teddy Bear #5731.

As a Round 1 NFT, ranked 162 and placed in Tier 3, #5731 earns Alice a total of 56 TEDY tokens. The calculation below confirms these results, revealing Alice’s well-deserved rewards. Since #5731 is her only NFT, her total TEDY rewards amount to 56 tokens.

In [ ]:
printfn "Round 1 NFT Total Rewards: %d" totalTedyRound1
printfn "Round 2 NFT Total Rewards: %d" totalTedyRound2
printfn "Total TEDY Rewards: %d" totalTedyReward

Round 1 NFT Total Rewards: 56
Round 2 NFT Total Rewards: 0
Total Tedy Rewards: 56


#### Disclaimers
The calculations above `marketcapAdjustment` are based on the initial market cap of 5,000,000,000, used to trace events during the INO event. The function adjusts the rewards to reflect the new market cap of 8,000,000, resulting in lower TEDY reward amounts. The figures provided are estimates and for informational purposes only. We do not guarantee their accuracy and are not liable for any investment decisions made based on this information. Use this information at your own risk.